# Init ptbxl

In [1]:
import numpy as np
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels as stats
import os

In [2]:
path = '../data/physionet.org/files/ptb-xl/1.0.3/ptbxl_database.csv'
df = pd.read_csv(path)
df.head()


,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
3,4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
4,5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


# Ziel Variable

In [3]:
df.columns

Index(['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse',
       'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')

In [4]:
print(f'Die Zielvariable hat {len(df["scp_codes"].unique())} unique values')

Die Zielvariable hat 5463 unique values


In [5]:
df['scp_codes'].head()

0    {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
1                {'NORM': 80.0, 'SBRAD': 0.0}
2                  {'NORM': 100.0, 'SR': 0.0}
3                  {'NORM': 100.0, 'SR': 0.0}
4                  {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, dtype: object

Die Zielvariable hat 5463 eindeutige Werte. Das liegt daran, dass sie in form von Dictionary ist (aber als String gespeichert ist), in dem viele verschiedene SCP-Codes enthalten sein können.

In [6]:
mi = ["IMI", "ASMI", "ILMI", "AMI", "ALMI", "INJAS", "LMI", "INJAL", "IPLMI", "IPMI", "INJIN", "INJLA", "PMI", "INJIL"]

In [19]:
counter = 0
for value in df['scp_codes']:
    for element in mi:
        if element in value:
            counter += 1
            # Ein Patient darf nur ein mal als mi klassifiziert werden
            break

print(f'Die Anzahl der kranken Menschen: {counter}')
print(f'Die Anzahl der gesunden Menschen: {df.shape[0] - counter}')
print(f'Prozent der kranken Menschen: {counter/df.shape[0] * 100:#.{5}}%')

Die Anzahl der kranken Menschen: 5469
Die Anzahl der gesunden Menschen: 16330
Prozent der kranken Menschen: 25.088%
